In [ ]:
import os
import cv2
import matplotlib
matplotlib.use('TkAgg')

import matplotlib.pyplot as plt
import numpy as np
import math


# zadanie 1
if not os.path.exists("knee.png") :
    !wget --no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/12_Segmentation/knee.png 

knee = cv2.imread('knee.png',cv2.IMREAD_GRAYSCALE)




# print(knee.shape)

x = 400
y = 250

def growing(img,x,y):
  img=cv2.GaussianBlur(img,(3,3),5)

  height,width = img.shape

  visited = np.zeros((height,width))
  segmented =  np.zeros((height,width))


  stack = []

  stack.append((y,x))
  
  
  
  visited[y,x] = 1
  segmented[y,x] = img[y,x]


  thr=45
  nS=0
  
  mV = img[y,x]
  while len(stack)>0 :
    px = stack.pop(0)
    nS += 1

    if px[0]+1 >= height or px[0]-1 <= 0 or px[1]+1 >= width or px[1]-1 <= 0 :
      continue
    mV = (mV * (nS-1) + img[px[0],px[1]]) / nS
    for i in range(px[0]-1,px[0]+2):
      for j in range(px[1]-1,px[1]+2):
        if i==0 and j==0:
          continue
        
        
        diff = abs(mV - img[i,j])

        if diff < thr and visited[i,j]==0:
          segmented[i,j] = img[i,j]
          stack.append((i,j))
        visited[i,j]=1
  
  return segmented

                                     
                                              
# plt.imshow(knee,'gray')
# plt.show()                      

# plt.imshow(growing(knee,600,400),'gray') 
# plt.show() 
  
# plt.imshow(growing(knee,264,287),'gray') 
# plt.show()




# zadanie 2

if not os.path.exists("umbrella.png") :
    !wget --no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/12_Segmentation/umbrella.png 



umbrella = cv2.imread('umbrella.png')
umbrella = cv2.cvtColor(umbrella,cv2.COLOR_BGR2HSV)
umbrella =umbrella[:,:,1]
# umbrella=cv2.GaussianBlur(umbrella,(3,3),5)
plt.imshow(umbrella,'gray')
plt.show()

minsize=3
segRes = np.zeros(umbrella.shape)
MRes = np.zeros(umbrella.shape)
index = 1 
thr = 5/255


def split(img,y1,y2,x1,x2):
  global index,segRes,MRes
  
  w=img[y1:y2,x1:x2]
  m=np.mean(w)
  
  st=np.std(w)
  s1 = y2-y1
  s2 = x2-x1

  if st > thr and s1> minsize and s2>minsize :
    y= math.floor((y1+y2)/2)
    x= math.floor((x1+x2)/2)

    split(img,y1,y,x1,x)
    split(img,y+1,y2,x1,x)
    split(img,y1,y,x+1,x2)
    split(img,y+1,y2,x+1,x2)

  else:
    segRes[y1:y2,x1:x2] = index
    MRes[y1:y2,x1:x2]=m
    index += 1
    

split(umbrella,1,umbrella.shape[0],1,umbrella.shape[1])

i=0
while i < index:
  ib = segRes==i
  ib = ib.astype('uint8')
  if not np.any(ib) :
    i +=1
    continue
  
  (y,x) = np.nonzero(ib)
  first = (y[0],x[0])
  
  kernel=cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
  # print(type(ib))
  dil = cv2.dilate(ib, kernel)
  mult = abs(dil-ib)
  mult_nz = mult[np.nonzero(mult)]
  mult_uniq = np.unique(mult_nz)

  joined =0

  for k in range(mult_uniq.size):
    ibs = (segRes==mult_uniq[k])
    (yf,xf) = np.nonzero(ib)
    ff = (yf[0],xf[0])

    diff = abs(MRes[y,x]-MRes[yf,xf])
    # print(MRes[y,x])
    # print(MRes[yf,xf])
    # print(diff)

    if np.any(diff < thr):
      segRes[ibs]=i
      joined=1

  if joined == 1:
    i +=1




unq = np.unique(segRes)
for i in range(unq.size):
  mask = (segRes==unq[i])

  if np.sum(mask)<23:
    segRes[mask] = 0

unq2 = np.unique(segRes)
segRes2 = segRes

for i in range(unq2.size):
  mask = (segRes2 == unq2[i])

  segRes[mask]=i

img2 = segRes











plt.imshow(img2,'gray')
plt.show()
plt.imshow(segRes,'gray')
plt.show()
plt.imshow(MRes,'gray')
plt.show()